<a href="https://colab.research.google.com/github/LouisStefanuto/Detection-of-the-PIK3CA-mutation-in-breast-cancer/blob/dirty-do-not-trust/attention_model_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, you will need to install some packages to reproduce the baseline.

In [1]:
!pip install tqdm
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pathlib import Path
from tqdm import tqdm

import logging

import numpy as np
import pandas as pd

import torch

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [3]:
# import data
PATH_COLAB = '/content/drive/MyDrive/challenge_ens_2023_small/moco_features.zip'
PATH_DEVICE = '..'
try:
    from google.colab import drive
    logging.info('Working on Colab.')
    
    # connect your drive to the session
    drive.mount('/content/drive')

    %cd /content/drive/MyDrive/challenge_data_ens_small/

    # unzip data into the colab session
    ! unzip $PATH_COLAB -d /content
    logging.info('Data unziped in your Drive.')

    %cd /content

    %cp -R drive/MyDrive/challenge_ens_2023_small/supplementary_data/ .
    %cp drive/MyDrive/challenge_ens_2023_small/train_output.csv .


except:
    logging.info('Working on your device.')
    
    data_exists = os.path.exists(PATH_DEVICE + '/train_input') and os.path.exists(PATH_DEVICE + '/test_input') and os.path.exists(PATH_DEVICE + '/train_output.csv')
    
    if data_exists:
        logging.info(f"Dataset found on device at : '{PATH_DEVICE}.'") 
    else:
        raise FileNotFoundError(f"Data folder not found at '{PATH_DEVICE}'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/challenge_data_ens_small/'
/content
Archive:  /content/drive/MyDrive/challenge_ens_2023_small/moco_features.zip
replace /content/test_input/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
/content


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Data architecture

After downloading or unzipping the downloaded files, your data tree must have the following architecture in order to properly run the notebook:
```
your_data_dir/
├── train_output.csv
├── train_input/
│   ├── images/
│       ├── ID_001/
│           ├── ID_001_tile_000_17_170_43.jpg
...
│   └── moco_features/
│       ├── ID_001.npy
...
├── test_input/
│   ├── images/
│       ├── ID_003/
│           ├── ID_003_tile_000_16_114_93.jpg
...
│   └── moco_features/
│       ├── ID_003.npy
...
├── supplementary_data/
│   ├── baseline.ipynb
│   ├── test_metadata.csv
│   └── train_metadata.csv
```

For instance, `your_data_dir = /storage/DATA_CHALLENGE_ENS_2022/`


This notebook aims to reproduce the baseline method on this challenge called `MeanPool`. This method consists in a logistic regression learnt on top of tile-level MoCo V2 features averaged over the slides.

For a given slide $s$ with $N_s=1000$ tiles and corresponding MoCo V2 features $\mathbf{K_s} \in \mathbb{R}^{(1000,\,2048)}$, a slide-level average is performed over the tile axis.

For $j=1,...,2048$:

$$\overline{\mathbf{k_s}}(j) = \frac{1}{N_s} \sum_{i=1}^{N_s} \mathbf{K_s}(i, j) $$

Thus, the training input data for MeanPool consists of $S_{\text{train}}=344$ mean feature vectors $\mathbf{k_s}$, $s=1,...,S_{\text{train}}$, where $S_{\text{train}}$ denotes the number of training samples.

## Data loading

In [5]:
# put your own path to the data root directory (see example in `Data architecture` section)
data_dir = Path("./")

# load the training and testing data sets
train_features_dir = data_dir / "train_input" / "moco_features"
test_features_dir = data_dir / "test_input" / "moco_features"
df_train = pd.read_csv(data_dir  / "supplementary_data" / "train_metadata.csv")
df_test = pd.read_csv(data_dir  / "supplementary_data" / "test_metadata.csv")

# concatenate y_train and df_train
y_train = pd.read_csv(data_dir  / "train_output.csv")
df_train = df_train.merge(y_train, on="Sample ID")

print(f"Training data dimensions: {df_train.shape}")  # (344, 4)
df_train.head()

Training data dimensions: (344, 4)


,Sample ID,Patient ID,Center ID,Target
0,ID_001.npy,P_001,C_1,0
1,ID_002.npy,P_002,C_2,1
2,ID_005.npy,P_005,C_5,0
3,ID_006.npy,P_006,C_5,0
4,ID_007.npy,P_007,C_2,1


## Data processing

We now load the features matrices $\mathbf{K_s} \in \mathbb{R}^{(1000,\,2048)}$ for $s=1,...,344$ and perform slide-level averaging. This operation should take at most 5 minutes on your laptop.

In [6]:
#size_train = 50 #len(df_train)
#X_train = np.zeros((size_train, 1000, 2048))
#y_train = np.zeros((size_train), dtype=int)
centers_train = []
patients_train = []

for i, (sample, label, center, patient) in enumerate(tqdm(
    df_train[["Sample ID", "Target", "Center ID", "Patient ID"]].values
)):
    # load the coordinates and features (1000, 3+2048)
    #_features = np.load(train_features_dir / sample)
    # get coordinates (zoom level, tile x-coord on the slide, tile y-coord on the slide)
    # and the MoCo V2 features
    #coordinates, features = _features[:, :3], _features[:, 3:]  # Ks
    # slide-level averaging
    #X_train[i] = features
    #y_train[i] = label
    centers_train.append(center)
    patients_train.append(patient)
    

# convert to numpy arrays
#X_train = np.array(X_train)
#y_train = np.array(y_train)
centers_train = np.array(centers_train)
patients_train = np.array(patients_train)

100%|██████████| 344/344 [00:00<00:00, 499944.76it/s]


## ROC-STAR loss

In [7]:
def roc_star_loss( _y_true, y_pred, gamma, _epoch_true, epoch_pred):
    """
    Nearly direct loss function for AUC.
    See article,
    C. Reiss, "Roc-star : An objective function for ROC-AUC that actually works."
    https://github.com/iridiumblue/articles/blob/master/roc_star.md
        _y_true: `Tensor`. Targets (labels).  Float either 0.0 or 1.0 .
        y_pred: `Tensor` . Predictions.
        gamma  : `Float` Gamma, as derived from last epoch.
        _epoch_true: `Tensor`.  Targets (labels) from last epoch.
        epoch_pred : `Tensor`.  Predicions from last epoch.
    """
    #convert labels to boolean
    y_true = (_y_true>=0.50)
    epoch_true = (_epoch_true>=0.50)

    # if batch is either all true or false return small random stub value.
    if torch.sum(y_true)==0 or torch.sum(y_true) == y_true.shape[0]: return torch.sum(y_pred)*1e-8

    pos = y_pred[y_true]
    neg = y_pred[~y_true]

    epoch_pos = epoch_pred[epoch_true]
    epoch_neg = epoch_pred[~epoch_true]

    # Take random subsamples of the training set, both positive and negative.
    max_pos = 1000 # Max number of positive training samples
    max_neg = 1000 # Max number of positive training samples
    cap_pos = epoch_pos.shape[0]
    cap_neg = epoch_neg.shape[0]
    epoch_pos = epoch_pos[torch.rand_like(epoch_pos) < max_pos/cap_pos]
    epoch_neg = epoch_neg[torch.rand_like(epoch_neg) < max_neg/cap_pos]

    ln_pos = pos.shape[0]
    ln_neg = neg.shape[0]

    # sum positive batch elements agaionst (subsampled) negative elements
    if ln_pos>0 :
        pos_expand = pos.view(-1,1).expand(-1,epoch_neg.shape[0]).reshape(-1)
        neg_expand = epoch_neg.repeat(ln_pos)

        diff2 = neg_expand - pos_expand + gamma
        l2 = diff2[diff2>0]
        m2 = l2 * l2
        len2 = l2.shape[0]
    else:
        m2 = torch.tensor([0], dtype=torch.float).cuda()
        len2 = 0

    # Similarly, compare negative batch elements against (subsampled) positive elements
    if ln_neg>0 :
        pos_expand = epoch_pos.view(-1,1).expand(-1, ln_neg).reshape(-1)
        neg_expand = neg.repeat(epoch_pos.shape[0])

        diff3 = neg_expand - pos_expand + gamma
        l3 = diff3[diff3>0]
        m3 = l3*l3
        len3 = l3.shape[0]
    else:
        m3 = torch.tensor([0], dtype=torch.float).cuda()
        len3=0

    if (torch.sum(m2)+torch.sum(m3))!=0 :
       res2 = torch.sum(m2)/max_pos+torch.sum(m3)/max_neg
       #code.interact(local=dict(globals(), **locals()))
    else:
       res2 = torch.sum(m2)+torch.sum(m3)

    res2 = torch.where(torch.isnan(res2), torch.zeros_like(res2), res2)

    return res2

In [8]:
def epoch_update_gamma(y_true,y_pred, epoch=-1,delta=2):
    """
    Calculate gamma from last epoch's targets and predictions.
    Gamma is updated at the end of each epoch.
    y_true: `Tensor`. Targets (labels).  Float either 0.0 or 1.0 .
    y_pred: `Tensor` . Predictions.
    """
    DELTA = delta
    SUB_SAMPLE_SIZE = 2000.0
    pos = y_pred[y_true==1]
    neg = y_pred[y_true==0] # yo pytorch, no boolean tensors or operators?  Wassap?
    # subsample the training set for performance
    cap_pos = pos.shape[0]
    cap_neg = neg.shape[0]
    pos = pos[torch.rand_like(pos) < SUB_SAMPLE_SIZE/cap_pos]
    neg = neg[torch.rand_like(neg) < SUB_SAMPLE_SIZE/cap_neg]
    ln_pos = pos.shape[0]
    ln_neg = neg.shape[0]
    pos_expand = pos.view(-1,1).expand(-1,ln_neg).reshape(-1)
    neg_expand = neg.repeat(ln_pos)
    diff = neg_expand - pos_expand
    ln_All = diff.shape[0]
    Lp = diff[diff>0] # because we're taking positive diffs, we got pos and neg flipped.
    ln_Lp = Lp.shape[0]-1
    diff_neg = -1.0 * diff[diff<0]
    diff_neg = diff_neg.sort()[0]
    ln_neg = diff_neg.shape[0]-1
    ln_neg = max([ln_neg, 0])
    left_wing = int(ln_Lp*DELTA)
    left_wing = max([0,left_wing])
    left_wing = min([ln_neg,left_wing])
    default_gamma=torch.tensor(0.2, dtype=torch.float).cuda()
    if diff_neg.shape[0] > 0 :
       gamma = diff_neg[left_wing]
    else:
       gamma = default_gamma # default=torch.tensor(0.2, dtype=torch.float).cuda() #zoink
    L1 = diff[diff>-1.0*gamma]
    ln_L1 = L1.shape[0]
    if epoch > -1 :
        return gamma
    else :
        return default_gamma

## Attention model

In [32]:
from torch import nn
from torch.functional import F

class AttentionMIL(nn.Module):
    def __init__(self):
        super(AttentionMIL, self).__init__()
        self.L = 2048
        self.D = 256
        self.K = 16

        self.feature_extractor_q = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.feature_extractor_k = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.attention_k = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        q = self.feature_extractor_q(x)  # BxNxK

        k = self.feature_extractor_k(x) #BxNxK
        kt = torch.transpose(k, 1, 2)  # BxKxN

        A = F.softmax(q@kt, dim=1)  # BxNxN

        M = A@x  # (BxNxN) (BxNxK) -> (BxNxK)

        M = torch.mean(M, dim=1)

        Y_prob = self.classifier(M)
        return Y_prob.squeeze(1)

In [33]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, df, features_dir, test = False):
        self.df = df
        self.features_dir = features_dir
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.test:
            sample = self.df.iloc[idx]["Sample ID"]
        else:
            sample, label = self.df.iloc[idx][["Sample ID", "Target"]]
            
        # load the coordinates and features (1000, 3+2048)
        _features = np.load(self.features_dir / sample)
        # get coordinates (zoom level, tile x-coord on the slide, tile y-coord on the slide)
        # and the MoCo V2 features
        coordinates, features = _features[:, :3], _features[:, 3:]  # Ks

        if self.test:
            return {'x': features}
        return {'x':features, 'y':label}

In [34]:
N_split = 250
train_dataset = MyDataset(df=df_train.iloc[:N_split], features_dir=train_features_dir)
val_dataset = MyDataset(df=df_train.iloc[N_split:], features_dir=train_features_dir)


In [35]:
from torch.utils.data import DataLoader


loader = DataLoader(dataset=train_dataset, batch_size=32)
val_loader = DataLoader(dataset=val_dataset, batch_size=32)



In [36]:
from torch.nn import BCELoss
from torch.optim import Adam

model = AttentionMIL().to(device)
criterion = BCELoss().to(device)
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=2)

In [37]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.last_best_val = 1e12
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if validation_loss >= self.last_best_val:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True
        else :
            self.counter = 0
            self.last_best_val = validation_loss

class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's 
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, best_valid_loss=float('inf'), save_path='outputs/best_model.pth'
    ):
        self.best_valid_loss = best_valid_loss
        self.save_path=save_path
        
    def __call__(
        self, current_valid_loss, 
        epoch, model, optimizer, criterion
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, self.save_path)

In [38]:
def train_fold(model, optimizer, criterion, loader, val_loader=[], n_epoch = 10, verbose=1, sigma=1e-2, model_name='model'):
    early_stopping = EarlyStopping(tolerance = 10, min_delta=0)
    save_best_model = SaveBestModel(save_path='./outputs_'+model_name+'_best_weights.pth')
    
    last_epoch_y_pred = torch.tensor( 1.0-np.random.rand(len(loader.dataset))/2.0, dtype=torch.float).to(device)

    last_epoch_y_t    = torch.tensor(np.concatenate([o['y'].float() for o in loader]),dtype=torch.float).to(device)
    epoch_gamma = 0.20
    for epoch in range(n_epoch):
        mean_loss = 0
        epoch_y_pred=[]
        epoch_y_t=[]
        for i, val in enumerate(loader):
            labels = val['y'].to(device)
            x = val['x'].to(device)
            
            noise = sigma * torch.randn_like(x)
            x = x + noise
            #print(labels.shape)
            optimizer.zero_grad()
            output = model(x)
            #print(output.shape)
            loss = roc_star_loss(labels.float(), output, epoch_gamma, last_epoch_y_t, last_epoch_y_pred)
            loss.backward()
            optimizer.step()
            mean_loss = 1/(i+1) * (loss - mean_loss) + i / (i+1) * mean_loss

            epoch_y_pred.extend(output)
            epoch_y_t.extend(labels.float())
        

        val_loss = 0
        for i, val in enumerate(val_loader):
            model.eval()

            labels = val['y'].to(device)
            x = val['x'].to(device)
            
            output = model(x)
            
            v_loss = criterion(output, labels.float())

            val_loss = 1/(i+1) * (v_loss - val_loss) + i / (i+1) * val_loss
        
        last_epoch_y_pred = torch.tensor(epoch_y_pred).to(device)
        last_epoch_y_t = torch.tensor(epoch_y_t).to(device)
        epoch_gamma = epoch_update_gamma(last_epoch_y_t, last_epoch_y_pred, epoch)
        if verbose:
            print(f"Epoch {epoch} - loss {mean_loss:.4f} - val loss {val_loss:.4f}")
        
        #best model
        save_best_model(val_loss, epoch=epoch, model=model, optimizer=optimizer, criterion=criterion)
        # early stopping
        early_stopping(loss, val_loss)
        if early_stopping.early_stop:
          print("We are at epoch:", epoch)
          del last_epoch_y_pred, last_epoch_y_t
          break

In [40]:
train_fold(model, optimizer, criterion, loader, val_loader, n_epoch=200, verbose=1, sigma = 0.01)

Epoch 0 - loss 0.1751 - val loss 0.3509

Best validation loss: 0.3508898913860321

Saving best model for epoch: 1

Epoch 1 - loss 0.0001 - val loss 0.3498

Best validation loss: 0.34976857900619507

Saving best model for epoch: 2

Epoch 2 - loss 0.0001 - val loss 0.3491

Best validation loss: 0.34913206100463867

Saving best model for epoch: 3

Epoch 3 - loss 0.0000 - val loss 0.3489

Best validation loss: 0.34888550639152527

Saving best model for epoch: 4

Epoch 4 - loss 0.0000 - val loss 0.3487

Best validation loss: 0.34868738055229187

Saving best model for epoch: 5



KeyboardInterrupt: ignored

In [41]:
# load the best model checkpoint
best_model_cp = torch.load('outputs_model_best_weights.pth')
best_model_epoch = best_model_cp['epoch']
print(f"Best model was saved at {best_model_epoch} epochs\n")

Best model was saved at 5 epochs



In [42]:
best_model_cp = torch.load('outputs_model_best_weights.pth')
model.load_state_dict(best_model_cp['model_state_dict'])

<All keys matched successfully>

In [43]:
y_trues = []
preds = []
for val in val_loader:
    model.eval()
    y_true = val['y'].detach().numpy().tolist()
    x = val['x'].to(device)

    pred = model(x).detach().cpu().numpy().tolist()
    y_trues.extend(y_true)
    preds.extend(pred)

auc = roc_auc_score(y_trues, preds)

print(auc)

0.4106800766283525


## 5-fold cross validation

In [20]:
# /!\ we perform splits at the patient level so that all samples from the same patient
# are found in the same split

patients_unique = np.unique(patients_train)
y_unique = np.array(
    [np.mean(y_train[patients_train == p]) for p in patients_unique]
)
centers_unique = np.array(
    [centers_train[patients_train == p][0] for p in patients_unique]
)

print(
    "Training set specifications\n"
    "---------------------------\n"
    f"{len(df_train)} unique samples\n"
    f"{len(patients_unique)} unique patients\n"
    f"{len(np.unique(centers_unique))} unique centers"
)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3438: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Training set specifications
---------------------------
344 unique samples
305 unique patients
3 unique centers


In [21]:
aucs = []
models = []
# 5-fold CV is repeated 5 times with different random states
for k in range(5):
    kfold = StratifiedKFold(5, shuffle=True, random_state=k)
    fold = 0
    # split is performed at the patient-level
    for train_idx_, val_idx_ in kfold.split(patients_unique, y_unique):
        # retrieve the indexes of the samples corresponding to the
        # patients in `train_idx_` and `test_idx_`
        train_idx = np.arange(len(df_train))[
            pd.Series(patients_train).isin(patients_unique[train_idx_])
        ]
        val_idx = np.arange(len(df_train))[
            pd.Series(patients_train).isin(patients_unique[val_idx_])
        ]
        # set the training and validation folds
        df_fold_train = df_train.iloc[train_idx]
        data_fold_train = MyDataset(df_fold_train, train_features_dir)
        loader_fold_train = DataLoader(data_fold_train, batch_size=40, shuffle=True)

        df_fold_val = df_train.iloc[val_idx]
        data_fold_val = MyDataset(df_fold_val, train_features_dir)
        loader_fold_val = DataLoader(data_fold_val, batch_size=40, shuffle=True)
        
        # instantiate the model
        model = AttentionMIL().to(device)
        criterion = BCELoss().to(device)
        optimizer = Adam(model.parameters(), lr=0.0005, weight_decay=1e-3)
        
        model_name = f'Attention_K{k}_F{fold}'
        train_fold(model, optimizer, criterion, loader_fold_train, val_loader=loader_fold_val, n_epoch=200, verbose=0, sigma = 0.001, model_name=model_name)

        best_model_cp = torch.load('outputs_'+ model_name + '_best_weights.pth')
        model.load_state_dict(best_model_cp['model_state_dict'])
        
        # get the predictions (1-d probability)
        y_trues = []
        preds = []
        for val in loader_fold_val:
            model.eval()
            y_true = val['y'].detach().cpu().numpy().tolist()
            x = val['x'].to(device)

            pred = model(x).detach().cpu().numpy().tolist()
            y_trues.extend(y_true)
            preds.extend(pred)

        auc = roc_auc_score(y_trues, preds)

        print(f"AUC on split {k} fold {fold}: {auc:.3f}")
        aucs.append(auc)
        # add the logistic regression to the list of classifiers
        models.append(model_name)

        del model
        fold += 1
    print("----------------------------")
print(
    f"5-fold cross-validated AUC averaged over {k+1} repeats: "
    f"{np.mean(aucs):.3f} ({np.std(aucs):.3f})"
)


Best validation loss: 0.6068194508552551

Saving best model for epoch: 1


Best validation loss: 0.5158516764640808

Saving best model for epoch: 2


Best validation loss: 0.49128687381744385

Saving best model for epoch: 3


Best validation loss: 0.45554786920547485

Saving best model for epoch: 6



KeyboardInterrupt: ignored

# Submission

Now we evaluate the previous models trained through cross-validation so that to produce a submission file that can directly be uploaded on the data challenge platform.

## Data processing

In [ ]:
df_test

In [ ]:
dataset_test = MyDataset(df_test, test_features_dir, test=True)

loader_test = DataLoader(dataset_test, batch_size=20, shuffle=False)

In [ ]:
dataset_test[1]

## Inference

In [ ]:
preds_test = 0


# loop over the classifiers
for model_name in models:
    model = AttentionMIL().to(device)
    best_model_cp = torch.load('outputs_'+ model_name + '_best_weights.pth')
    model.load_state_dict(best_model_cp['model_state_dict'])
    
    preds = []
    for val in loader_test:
        model.eval()
        x = val['x'].to(device)

        pred = model(x).detach().cpu().numpy().tolist()
        preds.extend(pred)

    preds = np.array(preds)
    preds_test += preds
# and take the average (ensembling technique)
preds_test = preds_test / len(models)

## Saving predictions

In [ ]:
submission = pd.DataFrame(
    {"Sample ID": df_test["Sample ID"].values, "Target": preds_test}
).sort_values(
    "Sample ID"
)  # extra step to sort the sample IDs

# sanity checks
assert all(submission["Target"].between(0, 1)), "`Target` values must be in [0, 1]"
assert submission.shape == (149, 2), "Your submission file must be of shape (149, 2)"
assert list(submission.columns) == [
    "Sample ID",
    "Target",
], "Your submission file must have columns `Sample ID` and `Target`"

# save the submission as a csv file
submission.to_csv(data_dir / "benchmark_test_output.csv", index=None)
submission.head()

# Dealing with images

The following code aims to load and manipulate the images provided as part of  this challenge.

## Scanning images paths on disk

This operation can take up to 5 minutes.

In [ ]:
train_images_dir = data_dir / "train_input" / "images"
train_images_files = list(train_images_dir.rglob("*.jpg"))

test_images_dir = data_dir / "test_input" / "images"
test_images_files = list(test_images_dir.rglob("*.jpg"))

print(
    f"Number of images\n"
    "-----------------\n"
    f"Train: {len(train_images_files)}\n" # 344 x 1000 = 344,000 tiles
    f"Test: {len(test_images_files)}\n"  # 149 x 1000 = 149,000 tiles
    f"Total: {len(train_images_files) + len(test_images_files)}\n"  # 493 x 1000 = 493,000 tiles
)

## Reading

Now we can load some of the `.jpg` images for a given sample, say `ID_001`.

In [ ]:
ID_001_tiles = [p for p in train_images_files if 'ID_001' in p.name]

In [ ]:
fig, axes = plt.subplots(5, 5)
fig.set_size_inches(12, 12)

for i, img_file in enumerate(ID_001_tiles[:25]):
    # get the metadata from the file path
    _, metadata = str(img_file).split("tile_")
    id_tile, level, x, y = metadata[:-4].split("_")
    img = plt.imread(img_file)
    ax = axes[i//5, i%5]
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Tile {id_tile} ({x}, {y})")
plt.show()

## Mapping with features

Note that the coordinates in the features matrices and tiles number are aligned.

In [ ]:
sample = "ID_001.npy"
_features = np.load(train_features_dir / sample)
coordinates, features = _features[:, :3], _features[:, 3:]
print("xy features coordinates")
coordinates[:10, 1:].astype(int)

In [ ]:
print(
    "Tiles numbering and features coordinates\n"
)
[tile.name for tile in ID_001_tiles[:10]]